## This notebook will combine the census blocks file with census data

In [1]:
import geopandas as gpd
import os
import getpass
import matplotlib.pyplot as plt
import re
import pandas as pd

In [19]:
# Find working directory, saves the path while appending the notebooks file name. 
# wd is now the file path to the park-equity-atx directory.
wd = os.getcwd()
wd = wd[:wd.find('notebooks')]
censusblocks = gpd.read_file(wd+"data/cbintersection/cbi.shp")
censustables = pd.read_excel(wd+"data/censustables/ACS_17_5yr_totalpop_allcounties.xlsx")

blocks = censusblocks.copy()
tables = censustables.copy()
print(blocks['GEOID10'].head)
print(tables['Id2'].head)
tables

<bound method NDFrame.head of 0        482090109011000
1        482090109011004
2        482090109081002
3        482090109011005
4        482090109011037
              ...       
10916    484910204092008
10917    484910205081009
10918    484910205101028
10919    484910204031006
10920    484910205093008
Name: GEOID10, Length: 10921, dtype: object>
<bound method NDFrame.head of 0      482090101001
1      482090101002
2      482090102001
3      482090102002
4      482090102003
           ...     
898    484910216013
899    484910216021
900    484910216022
901    484910216031
902    484910216032
Name: Id2, Length: 903, dtype: int64>


,Id,Id2,Estimate; Total
0,1500000US482090101001,482090101001,1197
1,1500000US482090101002,482090101002,1174
2,1500000US482090102001,482090102001,870
3,1500000US482090102002,482090102002,3914
4,1500000US482090102003,482090102003,1207
...,...,...,...
898,1500000US484910216013,484910216013,2590
899,1500000US484910216021,484910216021,1956
900,1500000US484910216022,484910216022,1111
901,1500000US484910216031,484910216031,667


In [20]:
blockgroup = blocks['GEOID10'].str.slice(start=0, stop=12)
blockgroup

tables['Id2'] = tables['Id2'].astype(str)

## Merge Dataframes:
According to the [documentation](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html) we have that the geocodes are each representing block data. Census tables will have a 12 digit geoID and the census blocks data will have a 15 digit block ID. Thus, in order to merge the files a new column will be created that will do a partial match.

In [21]:
# Convert the ID2 column to string data type. This will simplify the merge 
# by allowing pandas series regex commands. We will have to convert the 
# resulting column back...maybe. The GEOID in census 

merger = blocks.merge(tables, left_on=blockgroup, right_on='Id2', suffixes=('_blocks', '_tables'))

In [22]:
merger

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,FUNCSTAT10,...,jurisdic_2,jurisdic_3,modified_f,objectid,shape_area,shape_leng,geometry,Id,Id2,Estimate; Total
0,48,209,010901,1000,482090109011000,Block 1000,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3056196.943745553 10035434.26038514,...",1500000US482090109011,482090109011,2205
1,48,209,010901,1004,482090109011004,Block 1004,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3060340.445818823 10029833.26098854,...",1500000US482090109011,482090109011,2205
2,48,209,010901,1005,482090109011005,Block 1005,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3063053.367280419 10031649.19700599,...",1500000US482090109011,482090109011,2205
3,48,209,010901,1037,482090109011037,Block 1037,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3068833.199734814 10026135.52516772,...",1500000US482090109011,482090109011,2205
4,48,209,010901,1038,482090109011038,Block 1038,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3068616.191943968 10028509.66138104,...",1500000US482090109011,482090109011,2205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10916,48,491,020411,1009,484910204111009,Block 1009,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3104721.799123466 10136634.05634473,...",1500000US484910204111,484910204111,2861
10917,48,491,020411,1000,484910204111000,Block 1000,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3111491.587660338 10139505.2244327, ...",1500000US484910204111,484910204111,2861
10918,48,491,020411,1020,484910204111020,Block 1020,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3106701.031385466 10136501.8663204, ...",1500000US484910204111,484910204111,2861
10919,48,491,020503,1044,484910205031044,Block 1044,G5040,None,None,S,...,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,"POLYGON ((3106985.843647187 10146695.24721131,...",1500000US484910205031,484910205031,2642


In [25]:
merger.to_file(wd+"data/block_groups/merge.shp")